In [6]:
import mne #for exploring,visualizing and analyzing neurophysiological data such as EEG
import pandas as pd #data analysis and manipulation tool
import os  #for interacting with operating system, os.path to interact with file system
import numpy as np
import datetime
import json #Javascript Object Notation.lightweight format for storing and transporting data
from sklearn.preprocessing import MinMaxScaler #useful to transform features by scaling each feature to a given range
import re # the functions in this module let you check if a particular string matches a given regular expression
          ##a regular expression is a sequence of characters, these characters are to be searched in a text

import seaborn as sns #data visualization library based on matplotlib
import matplotlib.pyplot as plt
import shap #used to explain output of machine learning model using Shapley values. an application of game theory
import xgboost as xgb #machine learning library for regression, classification and ranking problems

In [7]:
def read_eeg(root): #function to retreive data from vhdr file. This function is called later. the input arguement is in the cell
                    ##cell consisting the trials array.  
                    
    vhdr_fname=root+trial_day.split("/")[0]+"_"+trial_num.split("/")[0]+"_EEG01.vhdr" #the split function removes the / from the
    ##strings in the trails array. trial_day and trial_num is defined in the cell below trials.
                                 
    #define list of indices for non-eeg channels
    misc_list=[]               #there are in total 82 channels. 64EEG channels +18 non EEG channels
    for i in range(18):
        misc_list.append(i+64)
        
        
    #read raw data #read_raw_brainvision--->reader for brain vision EEG file
    raw=mne.io.read_raw_brainvision(vhdr_fname,misc=misc_list,preload=True,verbose=False) 
    raw.info['line_freq']=500. #signals a float number
    
    #set montage
    montage=mne.channels.make_standard_montage('easycap-M1') #name of montage to use
    raw.set_montage(montage,verbose=False)
    
    
    #set common average referance,to specify the desired referance for EEG. this can be an existing electrode or virtual channel
    raw.set_eeg_reference('average', projection=False,verbose=False)
    
    #create a pandas dataframe with eeg data
    eeg_data = pd.DataFrame(raw.get_data().transpose(), columns=raw.ch_names)
    
    # Temporarily drop T7 while it's broken
    #eeg_data = eeg_data.drop(columns=["T7"])
    
    return eeg_data

In [8]:
def eeg_times(root):
    
    vhdr_path = root + trial_day.split("/")[0] + "_" + trial_num.split("/")[0] + "_EEG01.vhdr"
    
    import re

    with open(vhdr_path, "r+") as file: #r+ is used for both reading and writing. no need to call file.close() with the
                                        ##with statement. with statement ensures proper acquisition and release of resources

       #read the file contents
       file_contents = file.read()  #reads all the contents of the file
       text_pattern = re.compile(re.escape("%")) # the term escape is used to make % ineffective and compile is to compile a 
                                                 # a regular expression
       file_contents = text_pattern.sub("", file_contents) #substitutes "" in te file contents
       file.seek(0) #changes the pointer or lets say location in the file
       file.truncate() #truncates file from the current index. which in this case is zero.
       file.write(file_contents) 
    

    with open(vhdr_path, "r") as r:
        vhdr_lines = r.readlines() #reads all the lines
    
    eeg_start = ""

    for i,j in enumerate(vhdr_lines):
        if 'Recording Start Time' in j:
            eeg_start = vhdr_lines[i+1]
    eeg_start = eeg_start.splitlines()[0] #.split(".")[0]

    eeg_start_dt = datetime.datetime.strptime(eeg_start,'%Y-%m-%d %H:%M:%S.%f')

    return eeg_start_dt

In [9]:
def tobii_times(root): #the function is to read tobii data i.e. the pupilometric data. Here only the start and end datetime
                       ##is extracted

    with open(root + 'segment.json','r') as f: 
        seg_data = json.load(f)

    seg_start = seg_data['seg_t_start']
    seg_end = seg_data['seg_t_stop']

    seg_start = seg_start.replace("T"," ").replace("+",".")
    seg_start_dt = datetime.datetime.strptime(seg_start, '%Y-%m-%d %H:%M:%S.%f')

    seg_end = seg_end.replace("T"," ").replace("+",".")
    seg_end_dt = datetime.datetime.strptime(seg_end, '%Y-%m-%d %H:%M:%S.%f')
    
    return seg_start_dt, seg_end_dt

In [10]:
def EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df):    #syncing the start time betwween the tobii start 
                                                                  ##and eeg start because eeg starts earlier        
        #eeg_start in the eeg_times() function
        #eeg_df, tobii_start and tobii_end is in the same cell as where root is defined. below the trial cell
    fs = 500    
    skip_eeg_data = round((tobii_start - eeg_start).total_seconds()*fs) # 500 is the frequency

    total_event_duration = (tobii_end - tobii_start).total_seconds()
    event_eeg_duration = total_event_duration*fs

    eeg_data = eeg_df.iloc[skip_eeg_data:int(skip_eeg_data + event_eeg_duration)].reset_index().drop('index', axis=1)
    eeg_data['dateTime'] = pd.date_range(start=tobii_start, end=tobii_end, periods=len(eeg_data))
    eeg_data.index = eeg_data['dateTime']
    return eeg_data

In [11]:
def getEyeData(lines, tobii_start, tobii_end):  # function to read the tobii data
    gd, pc, pdi, gp, gp3 = [[],[]],[[],[]],[[],[]],[],[]
    ts, status = [], []
    j = 0
    for i in lines:
        # if i['s'] == 0:
        if 'gidx' in i:
            ts.append(i['ts'])
            j += 1
            if 'eye' in i:
                if i['eye'] == 'left':
                    if 'gd' in i:
                        gd[0].append(i['gd'])
                    elif 'pc' in i:
                        pc[0].append(i['pc'])
                    elif 'pd' in i:
                        pdi[0].append(i['pd'])
                        if i['s'] == 0:
                            status.append(i['s'])
                        elif i['s'] != 0:
                            status.append(i['s'])     
                elif i['eye'] == 'right':
                    if 'gd' in i:
                        gd[1].append(i['gd'])
                    elif 'pc' in i:
                        pc[1].append(i['pc'])
                    elif 'pd' in i:
                        pdi[1].append(i['pd'])
            elif 'gp' in i:
                gp.append(i['gp'])
            elif 'gp3' in i:
                gp3.append(i['gp3'])
        # elif i['s'] != 0:
        #         continue
    
    gd_df_left = pd.DataFrame(gd[0])
    gd_df_right = pd.DataFrame(gd[1])
    pc_df_left = pd.DataFrame(pc[0])
    pc_df_right = pd.DataFrame(pc[1])
    pdi_df_left = pd.DataFrame(pdi[0])
    pdi_df_right = pd.DataFrame(pdi[1])
    pda = (pdi_df_left + pdi_df_right)/2
    pd_diff = pdi_df_left - pdi_df_right
    pd_diff_abs = abs(pdi_df_left - pdi_df_right)
    pup_dis = pc_df_left.iloc[:,0] - pc_df_right.iloc[:,0]
    gp_df = pd.DataFrame(gp)
    gp3_df = pd.DataFrame(gp3)
    ts_np = np.unique(np.array(ts))
    ts = pd.DataFrame(ts_np)
    time = pd.DataFrame(pd.date_range(start=tobii_start, end=tobii_end, periods=len(ts)))
    status_df = pd.DataFrame(status)
    # print(status_df)
    
    
    df_full = pd.concat([time, status_df, gd_df_left, gd_df_right, pc_df_left, pc_df_right, pdi_df_left, pdi_df_right, pda, gp_df, gp3_df, pup_dis, pd_diff, pd_diff_abs], axis=1, ignore_index=True)
    df_full.columns = ['dateTime','status_tobii', 'gd_lx', 'gd_ly', 'gd_lz', 'gd_rx', 'gd_ry', 'gd_rz', 'pc_lx', 'pc_ly', 'pc_lz', 'pc_rx', 'pc_ry', 'pc_rz', 'pd_l', 'pd_r', 
                       'pda','gp_x', 'gp_y', 'gp3_x','gp3_y','gp3_z','pup_dis', 'pd_diff', 'pd_diff_abs']

    
    return df_full

In [12]:
def readTobii(root, tobii_start, tobii_end):
    lines = []
    # tobii_data = root + 'livedata.json/livedata.json'
    for line in open(root + 'livedata.json/livedata.json', 'r'):
        lines.append(json.loads(line))

    eye_metrics = getEyeData(lines, tobii_start, tobii_end)
    eye_metrics = eye_metrics.set_index('dateTime')
    # clean_em = eye_metrics[eye_metrics['status'] == 0]
    
#     eye_ratio = pd.read_csv(root + "eyesstream.csv")
#     eye_ratio['dateTime'] = pd.date_range(start=tobii_start, end=tobii_end, periods=len(eye_ratio))
#     eye_ratio = eye_ratio.loc[:,['dateTime','ratio']].set_index('dateTime')
    
#     # sync eye_ratio and eye_metrics
#     eye_combined = pd.merge_asof(left=eye_ratio, right=eye_metrics, right_index=True, left_index=True, direction='nearest', tolerance=pd.Timedelta('0.02s'))
    
#     return eye_combined
    return eye_metrics

In [13]:
def getSensorData(root, sensor, start, end, location=None):
    
    # location is roof or trunk
    # sensor is the name of sensor
    trial_day_roof = trial_day + "data/roofTopMints/" + trial_day.split("_")[0] + "/" + trial_day.split("_")[1] + "/" + trial_day.split("_")[2]
    trial_day_trunk = trial_day + "data/trunkSensors/" + trial_day.split("_")[0] + "/" + trial_day.split("_")[1] + "/" + trial_day.split("_")[2]

    if location == "roof":
        df = pd.read_csv(trial_day_roof + "MINTS_001e0636e527_" + sensor + "_" + root.split("/")[0] + ".csv")
    elif location == "trunk":
        df = pd.read_csv(trial_day_trunk + "MINTS_001e0610c2e7_" + sensor + "_" + root.split("/")[0] + ".csv", error_bad_lines=False)
    elif sensor == "palas":
        df = pd.read_csv(trial_day + trial_day.split("/")[0] + "_all_bins.csv")
    
    df['dateTime'] = pd.to_datetime(df['dateTime'])
    df = df.loc[(df['dateTime'] > start) & (df['dateTime'] < end)].reset_index().drop('index', axis=1).set_index('dateTime')
                         
    return df

In [14]:
def scaleSensor(df,var,roll_window=None, aggregation=None):
    
    scale = MinMaxScaler() 
    dummy = pd.DataFrame()
    if roll_window and aggregation == "mean":
        dummy[var] = df[var].rolling(roll_window).mean()
    elif roll_window and aggregation == "sum":
        dummy[var] = df[var].rolling(roll_window).sum()
    else:
        dummy[var] = df[var]
    return scale.fit_transform(dummy[var].to_numpy().reshape(-1,1))

In [15]:
def syncSensorTobiiEEG(sensor_df, eyes_df, eeg_df):
    
    tol_1 = pd.Timedelta('0.02 second') # 100 Hz (or whatever that is min(eeg_HZ, eyes_HZ))
    eye_eeg = pd.merge_asof(left=eyes_df, right=eeg_df, right_index=True, left_index=True, direction='nearest', tolerance=tol_1)
    
    tol_2 = pd.Timedelta('1S')
    sensor_eye_eeg = pd.merge_asof(left=sensor_df, right=eye_eeg, right_index=True, left_index=True, direction='nearest', tolerance=tol_2).dropna()
    # print(list(sensor_eye_eeg.columns))
    sensor_eye_eeg = sensor_eye_eeg[sensor_eye_eeg['status_tobii'] == 0]
    sensor_eye_eeg = sensor_eye_eeg[sensor_eye_eeg['pc_rx'] != 0]
    
    return sensor_eye_eeg

In [16]:
trials = [
    ["2021_05_26/", "T03_U000/"],
    ["2021_05_26/", "T05_U000/"],
    ["2021_06_09/", "T03_U000/"],
    ["2021_06_09/", "T04_U000/"],
    ["2021_06_10/", "T03_U000/"],
    ["2021_06_10/", "T04_U000/"],
]


In [17]:
%%time

full_df = pd.DataFrame()

for i in trials:
    trial_day, trial_num, root = i[0], i[1], i[0] + i[1]
    eeg_start = eeg_times(root) 
    tobii_start, tobii_end = tobii_times(root)
    eeg_df_full = read_eeg(root)   #Will retrieve the eeg data
    eeg_df = EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df_full)
    tobii_data = readTobii(root, tobii_start, tobii_end, ) #Will retreive the tobii data
    
    # sensors
    
    # sensor_df = getSensorData(root, 'MGS001', tobii_start, tobii_end, 'roof')
    #sensor_df = getSensorData(root, 'palas', tobii_start, tobii_end)
    # print("Parsing " + root)
    sensor_df = getSensorData(root, '2B-NOX', tobii_start, tobii_end, 'trunk')
    
    synced_full_df = syncSensorTobiiEEG(sensor_df, tobii_data, eeg_df)
    full_df = pd.concat([full_df,synced_full_df])
    

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


b'Skipping line 16122: expected 14 fields, saw 18\n'
The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


b'Skipping line 16122: expected 14 fields, saw 18\n'
The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.




Wall time: 39 s


The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.




In [18]:
pd.set_option('display.max_columns', None)
eeg_df

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,0.001702,0.004508,-0.008591,-0.004093,-0.000593,0.002736,0.006220,0.004985,0.003791,0.000327,0.015805,-0.000413,0.001278,-0.001994,-0.006885,-0.002444,-0.000833,0.002420,-0.002364,-0.000941,0.002599,0.000066,-0.003732,0.008034,-0.002318,-0.000082,-0.002074,-0.001163,0.002574,-0.000088,0.003863,-0.002251,-0.002338,-0.002045,0.002106,0.000353,0.002081,0.004246,0.000054,0.002104,-0.008147,-0.002191,0.003456,0.004849,0.000031,0.003599,-0.000791,0.000939,0.001964,0.000076,0.000702,-0.001102,-0.001083,0.002184,-0.002757,-0.005135,-0.010631,-0.004571,-0.002086,-0.001366,0.000614,-0.000393,-0.004239,-0.000531,-0.052083,-0.052083,-0.052083,-0.052083,0.000809,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041909,-0.070300,-4073.660851,10860.770226,-3843.471050,19853.0,0.0,2021-06-10 11:21:49.000000000
2021-06-10 11:21:49.002000012,0.001698,0.004494,-0.008588,-0.004090,-0.000590,0.002733,0.006221,0.004986,0.003783,0.000322,0.015812,-0.000423,0.001281,-0.001976,-0.006887,-0.002439,-0.000832,0.002422,-0.002364,-0.000951,0.002603,0.000069,-0.003728,0.008033,-0.002315,-0.000088,-0.002068,-0.001162,0.002573,-0.000073,0.003865,-0.002252,-0.002335,-0.002041,0.002109,0.000354,0.002082,0.004244,0.000055,0.002113,-0.008145,-0.002195,0.003459,0.004849,0.000026,0.003597,-0.000789,0.000946,0.001968,0.000070,0.000702,-0.001100,-0.001089,0.002144,-0.002754,-0.005147,-0.010635,-0.004556,-0.002090,-0.001366,0.000615,-0.000391,-0.004243,-0.000526,-0.052083,-0.052083,-0.052083,-0.052083,0.000833,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041881,-0.070267,-4073.660851,10825.892448,-3822.544575,19854.0,0.0,2021-06-10 11:21:49.002000012
2021-06-10 11:21:49.004000024,0.001694,0.004482,-0.008588,-0.004090,-0.000589,0.002735,0.006220,0.004988,0.003780,0.000324,0.015813,-0.000435,0.001272,-0.001961,-0.006888,-0.002432,-0.000832,0.002425,-0.002363,-0.000950,0.002604,0.000070,-0.003725,0.008035,-0.002314,-0.000089,-0.002067,-0.001157,0.002583,-0.000084,0.003866,-0.002254,-0.002332,-0.002041,0.002109,0.000356,0.002082,0.004243,0.000053,0.002118,-0.008145,-0.002196,0.003461,0.004852,0.000024,0.003597,-0.000788,0.000938,0.001969,0.000070,0.000700,-0.001096,-0.001097,0.002125,-0.002752,-0.005142,-0.010643,-0.004547,-0.002093,-0.001362,0.000615,-0.000388,-0.004245,-0.000514,-0.052083,-0.052083,-0.052083,-0.052083,0.000877,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041881,-0.070276,-4164.341450,10832.868576,-3822.544575,19855.0,0.0,2021-06-10 11:21:49.004000024
2021-06-10 11:21:49.006000036,0.001691,0.004489,-0.008589,-0.004097,-0.000592,0.002738,0.006217,0.004988,0.003786,0.000327,0.015808,-0.000442,0.001270,-0.001935,-0.006889,-0.002428,-0.000833,0.002429,-0.002364,-0.000947,0.002603,0.000069,-0.003726,0.008036,-0.002313,-0.000087,-0.002072,-0.001153,0.002568,-0.000092,0.003864,-0.002253,-0.002333,-0.002043,0.002108,0.000356,0.002081,0.004241,0.000051,0.002103,-0.008149,-0.002195,0.003459,0.004854,0.000025,0.003600,-0.000790,0.000931,0.001967,0.000072,0.000697,-0.001095,-0.001095,0.002154,-0.002754,-0.005130,-0.010649,-0.004548,-0.002088,-0.001357,0.000611,-0.000390,-0.004246,-0.000517,-0.052083,-0.052083,-0.052083,-0.052083,0.000912,35.716936,15856.999904,0.000098,96.0,11666.615486,29.7500,-0.041924,-0.070262,-4171.317101,10867.745399,-3829.519987,19856.0,0.0,2021-06-10 11:21:49.006000036
2021-06-10 11:21:49.008000049,0.001693,0.004494,-0.008589,-0.004100,-0.000593,0.002740,0.006213,0.004986,0.003780,0.000325,0.015805,-0.000435,0.001280,-0.001931,-0.006891,-

In [19]:
tobii_data

,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs
dateTime,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,0,0.0089,0.2535,0.9673,0.0601,0.2419,0.9684,31.18,-32.46,-32.70,-34.76,-30.32,-29.89,3.06,2.99,3.025,0.4831,0.2434,40.74,273.85,1161.22,65.94,0.07,0.07
2021-06-10 11:21:49.009975138,0,0.0112,0.2638,0.9645,0.0568,0.2527,0.9659,31.19,-32.68,-32.67,-34.82,-30.12,-29.94,3.10,3.03,3.065,0.4834,0.2286,44.95,324.11,1297.45,66.01,0.07,0.07
2021-06-10 11:21:49.019950277,0,0.0100,0.2627,0.9648,0.0576,0.2600,0.9639,31.17,-32.52,-32.66,-34.75,-30.36,-29.82,3.10,3.04,3.070,0.4835,0.2247,44.97,330.25,1303.15,65.92,0.06,0.06
2021-06-10 11:21:49.029925416,0,0.0115,0.2682,0.9633,0.0571,0.2613,0.9636,31.20,-32.72,-32.57,-34.82,-30.19,-29.80,3.09,3.02,3.055,0.4832,0.2202,47.05,345.25,1339.60,66.02,0.07,0.07
2021-06-10 11:21:49.039900555,0,0.0127,0.2708,0.9626,0.0566,0.2681,0.9617,31.24,-32.60,-32.78,-34.75,-30.41,-29.75,3.13,3.05,3.090,0.4829,0.2136,50.34,373.35,1414.36,65.99,0.08,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10 11:27:13.960099444,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00
2021-06-10 11:27:13.970074583,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00
2021-06-10 11:27:13.980049722,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00


In [20]:
synced_full_df

,NO2,NO,NOX,temperature,pressure,flow,O3-flow,voltage,O3-voltage,scrubber-temperature,error-byte,sensorTime,status,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:51.069614,7.0,18.2,25.3,43.0,937.1,1644,101.3,0.761633,0.099,110.5,0,"10/06/21,05:20:19",80,0,0.1265,0.4828,0.8665,0.1553,0.5095,0.8463,31.45,-31.88,-32.73,-33.88,-29.35,-29.79,3.22,3.25,3.235,0.0000,0.0000,0.00,0.00,0.00,65.33,-0.03,0.03,0.001734,0.004505,-0.008615,-0.004258,-0.000582,0.002733,0.006248,0.005046,0.003798,0.000248,0.015842,-0.000622,0.001303,-0.001938,-0.006895,-0.002480,-0.000798,0.002431,-0.002375,-0.000959,0.002582,0.000038,-0.003774,0.007953,-0.002235,-0.000053,-0.002012,-0.001061,0.002518,-0.000387,0.003852,-0.002172,-0.002258,-0.002036,0.002110,0.000378,0.002096,0.004238,0.000095,0.002114,-0.008113,-0.002235,0.003447,0.004897,0.000084,0.003631,-0.000766,0.000668,0.002068,0.000177,0.000814,-0.000935,-0.001111,0.002236,-0.002642,-0.005112,-0.010648,-0.004562,-0.002070,-0.001382,0.000604,-0.000521,-0.004272,-0.000609,-0.052083,-0.052083,-0.052083,-0.052083,0.000759,35.671089,16958.000138,0.000098,96.0,11677.211761,29.7500,-0.042057,-0.070124,-5203.682899,10749.162674,-3390.066862,20885.0,0.0,2021-06-10 11:21:51.064012701
2021-06-10 11:21:56.067877,7.0,17.7,24.7,43.0,937.5,1620,100.5,0.761654,0.102,110.5,0,"10/06/21,05:20:24",80,0,-0.0869,0.5598,0.8240,-0.1066,0.6505,0.7520,30.29,-32.05,-31.33,-35.02,-29.99,-27.97,3.47,3.49,3.480,0.0000,0.0000,0.00,0.00,0.00,65.31,-0.02,0.02,0.001576,0.004344,-0.008273,-0.003618,-0.000881,0.002393,0.006132,0.004926,0.003622,0.000256,0.015824,0.000118,0.001654,-0.001618,-0.006825,-0.002390,-0.000950,0.002125,-0.002551,-0.000949,0.002457,-0.000113,-0.003979,0.007697,-0.002410,-0.000286,-0.002260,-0.000989,0.002920,0.000741,0.003640,-0.002240,-0.002324,-0.002269,0.001924,0.000054,0.001993,0.004029,0.000113,0.002071,-0.008125,-0.002318,0.003222,0.004712,-0.000023,0.003706,-0.000746,0.001291,0.001860,0.000056,0.000721,-0.000803,-0.001203,0.002147,-0.002622,-0.004799,-0.010375,-0.004359,-0.002186,-0.001183,0.000435,-0.000554,-0.004468,-0.000068,-0.052083,-0.052083,-0.052083,-0.052083,0.000863,36.037885,16935.000196,0.000098,94.0,11680.744171,29.7500,-0.042090,-0.070176,-5998.883724,6229.073524,8147.321701,23389.0,0.0,2021-06-10 11:21:56.072043520
2021-06-10 11:22:01.062515,7.1,17.3,24.4,43.0,937.6,1642,99.9,0.761671,0.098,110.6,0,"10/06/21,05:20:29",80,0,-0.0837,0.3188,0.9441,-0.0795,0.3434,0.9358,30.59,-32.87,-31.41,-35.53,-30.10,-28.41,2.94,3.02,2.980,0.5649,0.1867,-33.92,96.44,333.55,66.12,-0.08,0.08,0.001316,0.004107,-0.008237,-0.003676,-0.001030,0.002279,0.005952,0.005030,0.003902,0.000508,0.015949,-0.000015,0.001259,-0.001785,-0.006692,-0.001621,-0.001156,0.001992,-0.002497,-0.000872,0.002404,-0.000105,-0.004126,0.007667,-0.002422,-0.000143,-0.002411,-0.000592,0.002572,0.000293,0.003344,-0.002207,-0.002259,-0.002203,0.001703,-0.000032,0.002046,0.004201,-0.000126,0.002366,-0.008079,-0.002356,0.003054,0.004611,-0.000027,0.003710,-0.000882,0.000855,0.001840,0.000039,0.000787,-0.000493,-0.001556,0.001938,-0.002794,-0.004760,-0.010593,-0.004118,-0.001734,-0.000551,0.000375,-0.000427,-0.004307,0.000783,-0.052083,-0.052083,-0.052083,-0.052083,0.001192,33.699550,16704.000533,0.000098,98.0,11666.615486,29.7500,-0.042062,-0.070052,-2580.915213,7847.377300,3557.477713,25883.0,0.0,2021-06-10 11:22:01.060074215
2021-06-10 11:22:06.068050,7.

In [22]:
pd.set_option('display.max_columns', None)
full_df

,NO2,NO,NOX,temperature,pressure,flow,O3-flow,voltage,O3-voltage,scrubber-temperature,error-byte,sensorTime,status,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-26 11:45:59.927799,12.4,-0.3,12.1,41.4,941.2,1613,100.9,0.883396,0.108,110.6,0,"26/05/21,05:45:02",80,0.0,-0.1213,0.2150,0.9690,0.0270,0.1731,0.9845,30.01,-33.14,-31.58,-35.23,-30.35,-27.96,4.11,4.20,4.155,0.5394,0.3570,-22.04,47.11,366.73,65.24,-0.09,0.09,-0.001457,0.016197,-0.004553,-0.024539,0.001305,0.002413,0.006121,0.005402,0.002292,0.000938,-0.000837,-0.013052,0.000281,-0.017478,0.000191,-0.015168,0.001843,0.005521,-0.002029,-0.002055,0.002172,0.002507,-0.001387,0.002925,0.000607,-0.020358,0.001359,0.000154,0.000247,-0.000252,0.007846,0.000945,-0.000653,-0.002045,0.004555,0.005464,0.007004,0.004779,-0.003496,-0.005260,-0.001633,-0.009553,0.006312,0.007187,0.002384,-0.004333,0.001982,-0.004955,0.005255,-0.000981,0.005001,0.002947,-0.001608,0.004212,0.001255,-0.001919,0.035828,-0.005215,-0.001422,-0.001484,-0.001545,0.001445,0.001109,-0.008719,-0.052083,-0.052083,-0.052083,-0.052083,-0.000408,46.216525,16319.999471,0.000100,84.0,12482.537270,30.1875,-0.037508,-0.043521,-4729.352474,10058.593750,-411.551327,17521.0,0.0,2021-05-26 11:45:59.924009126
2021-05-26 11:46:04.932462,12.6,-0.5,12.2,41.3,941.2,1635,100.4,0.883441,0.104,110.5,0,"26/05/21,05:45:07",80,0.0,-0.0629,0.2644,0.9624,0.0998,0.2427,0.9650,30.23,-33.24,-31.42,-34.99,-29.56,-28.06,3.58,3.47,3.525,0.4957,0.2853,4.81,69.11,352.29,65.22,0.11,0.11,-0.001563,0.016150,-0.004474,-0.025195,0.001241,0.002366,0.006094,0.005359,0.002197,0.000694,-0.001067,-0.013243,0.000247,-0.017507,0.000299,-0.015106,0.001951,0.005473,-0.001874,-0.002099,0.002117,0.002444,-0.001436,0.002894,0.000571,-0.020411,0.001264,0.000120,0.000380,-0.000186,0.007813,0.000957,-0.000750,-0.002087,0.004490,0.005445,0.006976,0.004721,-0.003469,-0.005184,-0.001747,-0.009553,0.006273,0.007156,0.002334,-0.004347,0.001775,-0.004877,0.005203,-0.001040,0.004971,0.002903,-0.001627,0.004099,0.001155,-0.001876,0.038740,-0.005227,-0.001397,-0.001828,-0.001638,0.001358,0.001071,-0.008492,-0.052083,-0.052083,-0.052083,-0.052083,-0.000526,44.886883,16416.000202,0.000099,87.0,12524.922371,30.1875,-0.037551,-0.043554,-6277.901649,10260.881424,753.348231,20024.0,0.0,2021-05-26 11:46:04.930018405
2021-05-26 11:46:09.932680,12.9,-0.6,12.2,41.3,941.2,1621,99.1,0.883443,0.105,110.5,0,"26/05/21,05:45:12",80,0.0,-0.0076,0.2236,0.9746,0.1600,0.2010,0.9664,30.55,-33.24,-31.91,-34.51,-29.98,-28.30,3.59,3.51,3.550,0.4564,0.3384,27.29,49.93,342.59,65.06,0.08,0.08,-0.001550,0.016242,-0.004641,-0.025350,0.001296,0.002435,0.006165,0.005412,0.002211,0.000762,-0.001122,-0.013340,0.000268,-0.017445,0.000264,-0.015030,0.001895,0.005560,-0.001921,-0.002089,0.002188,0.002507,-0.001362,0.002944,0.000566,-0.020392,0.001312,0.000223,0.000465,-0.000195,0.007862,0.000891,-0.000822,-0.002204,0.004537,0.005544,0.007088,0.004790,-0.003470,-0.005347,-0.001780,-0.009540,0.006329,0.007213,0.002414,-0.004324,0.001633,-0.004924,0.005239,-0.000992,0.005006,0.002960,-0.001671,0.004106,0.001140,-0.001971,0.037930,-0.005165,-0.001392,-0.001853,-0.001610,0.001439,0.001137,-0.008469,-0.052083,-0.052083,-0.052083,-0.052083,-0.000304,45.207832,16349.000856,0.000098,88.0,12567.308426,30.0625,-0.037513,-0.043559,-6591.796875,10044.642448,460.379452,22526.0,0.0,2021-05-26 11:46:09.934027681
2021-05-26 11:46:14.919361,13.1

In [23]:
full_psds_for_2021=pd.read_pickle("full_psds_for_2021.pkl")

In [24]:
full_psds_for_2021

,Fp1-delta ($\mathrm{V^2}$/Hz),Fp2-delta ($\mathrm{V^2}$/Hz),F3-delta ($\mathrm{V^2}$/Hz),F4-delta ($\mathrm{V^2}$/Hz),C3-delta ($\mathrm{V^2}$/Hz),C4-delta ($\mathrm{V^2}$/Hz),P3-delta ($\mathrm{V^2}$/Hz),P4-delta ($\mathrm{V^2}$/Hz),O1-delta ($\mathrm{V^2}$/Hz),O2-delta ($\mathrm{V^2}$/Hz),F7-delta ($\mathrm{V^2}$/Hz),F8-delta ($\mathrm{V^2}$/Hz),T7-delta ($\mathrm{V^2}$/Hz),T8-delta ($\mathrm{V^2}$/Hz),P7-delta ($\mathrm{V^2}$/Hz),P8-delta ($\mathrm{V^2}$/Hz),Fz-delta ($\mathrm{V^2}$/Hz),Cz-delta ($\mathrm{V^2}$/Hz),Pz-delta ($\mathrm{V^2}$/Hz),Oz-delta ($\mathrm{V^2}$/Hz),FC1-delta ($\mathrm{V^2}$/Hz),FC2-delta ($\mathrm{V^2}$/Hz),CP1-delta ($\mathrm{V^2}$/Hz),CP2-delta ($\mathrm{V^2}$/Hz),FC5-delta ($\mathrm{V^2}$/Hz),FC6-delta ($\mathrm{V^2}$/Hz),CP5-delta ($\mathrm{V^2}$/Hz),CP6-delta ($\mathrm{V^2}$/Hz),FT9-delta ($\mathrm{V^2}$/Hz),FT10-delta ($\mathrm{V^2}$/Hz),FCz-delta ($\mathrm{V^2}$/Hz),AFz-delta ($\mathrm{V^2}$/Hz),F1-delta ($\mathrm{V^2}$/Hz),F2-delta ($\mathrm{V^2}$/Hz),C1-delta ($\mathrm{V^2}$/Hz),C2-delta ($\mathrm{V^2}$/Hz),P1-delta ($\mathrm{V^2}$/Hz),P2-delta ($\mathrm{V^2}$/Hz),AF3-delta ($\mathrm{V^2}$/Hz),AF4-delta ($\mathrm{V^2}$/Hz),FC3-delta ($\mathrm{V^2}$/Hz),FC4-delta ($\mathrm{V^2}$/Hz),CP3-delta ($\mathrm{V^2}$/Hz),CP4-delta ($\mathrm{V^2}$/Hz),PO3-delta ($\mathrm{V^2}$/Hz),PO4-delta ($\mathrm{V^2}$/Hz),F5-delta ($\mathrm{V^2}$/Hz),F6-delta ($\mathrm{V^2}$/Hz),C5-delta ($\mathrm{V^2}$/Hz),C6-delta ($\mathrm{V^2}$/Hz),P5-delta ($\mathrm{V^2}$/Hz),P6-delta ($\mathrm{V^2}$/Hz),AF7-delta ($\mathrm{V^2}$/Hz),AF8-delta ($\mathrm{V^2}$/Hz),FT7-delta ($\mathrm{V^2}$/Hz),FT8-delta ($\mathrm{V^2}$/Hz),TP7-delta ($\mathrm{V^2}$/Hz),TP8-delta ($\mathrm{V^2}$/Hz),PO7-delta ($\mathrm{V^2}$/Hz),PO8-delta ($\mathrm{V^2}$/Hz),Fpz-delta ($\mathrm{V^2}$/Hz),CPz-delta ($\mathrm{V^2}$/Hz),POz-delta ($\mathrm{V^2}$/Hz),TP10-delta ($\mathrm{V^2}$/Hz),Fp1-theta ($\mathrm{V^2}$/Hz),Fp2-theta ($\mathrm{V^2}$/Hz),F3-theta ($\mathrm{V^2}$/Hz),F4-theta ($\mathrm{V^2}$/Hz),C3-theta ($\mathrm{V^2}$/Hz),C4-theta ($\mathrm{V^2}$/Hz),P3-theta ($\mathrm{V^2}$/Hz),P4-theta ($\mathrm{V^2}$/Hz),O1-theta ($\mathrm{V^2}$/Hz),O2-theta ($\mathrm{V^2}$/Hz),F7-theta ($\mathrm{V^2}$/Hz),F8-theta ($\mathrm{V^2}$/Hz),T7-theta ($\mathrm{V^2}$/Hz),T8-theta ($\mathrm{V^2}$/Hz),P7-theta ($\mathrm{V^2}$/Hz),P8-theta ($\mathrm{V^2}$/Hz),Fz-theta ($\mathrm{V^2}$/Hz),Cz-theta ($\mathrm{V^2}$/Hz),Pz-theta ($\mathrm{V^2}$/Hz),Oz-theta ($\mathrm{V^2}$/Hz),FC1-theta ($\mathrm{V^2}$/Hz),FC2-theta ($\mathrm{V^2}$/Hz),CP1-theta ($\mathrm{V^2}$/Hz),CP2-theta ($\mathrm{V^2}$/Hz),FC5-theta ($\mathrm{V^2}$/Hz),FC6-theta ($\mathrm{V^2}$/Hz),CP5-theta ($\mathrm{V^2}$/Hz),CP6-theta ($\mathrm{V^2}$/Hz),FT9-theta ($\mathrm{V^2}$/Hz),FT10-theta ($\mathrm{V^2}$/Hz),FCz-theta ($\mathrm{V^2}$/Hz),AFz-theta ($\mathrm{V^2}$/Hz),F1-theta ($\mathrm{V^2}$/Hz),F2-theta ($\mathrm{V^2}$/Hz),C1-theta ($\mathrm{V^2}$/Hz),C2-theta ($\mathrm{V^2}$/Hz),P1-theta ($\mathrm{V^2}$/Hz),P2-theta ($\mathrm{V^2}$/Hz),AF3-theta ($\mathrm{V^2}$/Hz),AF4-theta ($\mathrm{V^2}$/Hz),FC3-theta ($\mathrm{V^2}$/Hz),FC4-theta ($\mathrm{V^2}$/Hz),CP3-theta ($\mathrm{V^2}$/Hz),CP4-theta ($\mathrm{V^2}$/Hz),PO3-theta ($\mathrm{V^2}$/Hz),PO4-theta ($\mathrm{V^2}$/Hz),F5-theta ($\mathrm{V^2}$/Hz),F6-theta ($\mathrm{V^2}$/Hz),C5-theta ($\mathrm{V^2}$/Hz),C6-theta ($\mathrm{V^2}$/Hz),P5-theta ($\mathrm{V^2}$/Hz),P6-theta ($\mathrm{V^2}$/Hz),AF7-theta ($\mathrm{V^2}$/Hz),AF8-theta ($\mathrm{V^2}$/Hz),FT7-theta ($\mathrm{V^2}$/Hz),FT8-theta ($\mathrm{V^2}$/Hz),TP7-theta ($\mathrm{V^2}$/Hz),TP8-theta ($\mathrm{V^2}$/Hz),PO7-theta ($\mathrm{V^2}$/Hz),PO8-theta ($\mathrm{V^2}$/Hz),Fpz-theta ($\mathrm{V^2}$/Hz),CPz-theta ($\mathrm{V^2}$/Hz),POz-theta ($\mathrm{V^2}$/Hz),TP10-theta ($\mathrm{V^2}$/Hz),Fp1-alpha ($\mathrm{V^2}$/Hz),Fp2-alpha ($\mathrm{V^2}$/Hz),F3-alpha ($\mathrm{V^2}$/Hz),F4-alpha ($\mathrm{V^2}$/Hz),C3-alpha ($\mathrm{V^2}$/Hz),C4-alpha ($\mathrm{V^2}$/Hz),P3-alpha ($\mathrm{V^2}$/Hz),P4-alpha (

In [19]:
pd.set_option('display.max_columns', None)
tol_2 = pd.Timedelta('1S')
full_df_2021 = pd.merge_asof(left=full_df, right=full_psds_for_2021, right_index=True, left_index=True, direction='nearest', tolerance=tol_2).dropna()
full_df_2021

,NO2,NO,NOX,temperature,pressure,flow,O3-flow,voltage,O3-voltage,scrubber-temperature,error-byte,sensorTime,status,status_tobii,gd_lx,gd_ly,gd_lz,gd_rx,gd_ry,gd_rz,pc_lx,pc_ly,pc_lz,pc_rx,pc_ry,pc_rz,pd_l,pd_r,pda,gp_x,gp_y,gp3_x,gp3_y,gp3_z,pup_dis,pd_diff,pd_diff_abs,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,Oz,FC1,FC2,CP1,CP2,FC5,FC6,CP5,CP6,FT9,FT10,FCz,AFz,F1,F2,C1,C2,P1,P2,AF3,AF4,FC3,FC4,CP3,CP4,PO3,PO4,F5,F6,C5,C6,P5,P6,AF7,AF8,FT7,FT8,TP7,TP8,PO7,PO8,Fpz,CPz,POz,TP10,ExG 1,ExG 2,ExG 3,ExG 4,ECG.,Resp.,PPG,SpO2,HR,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime,Fp1_Delta,Fp2_Delta,F3_Delta,F4_Delta,C3_Delta,C4_Delta,P3_Delta,P4_Delta,O1_Delta,O2_Delta,F7_Delta,F8_Delta,T7_Delta,T8_Delta,P7_Delta,P8_Delta,Fz_Delta,Cz_Delta,Pz_Delta,Oz_Delta,FC1_Delta,FC2_Delta,CP1_Delta,CP2_Delta,FC5_Delta,FC6_Delta,CP5_Delta,CP6_Delta,FT9_Delta,FT10_Delta,FCz_Delta,AFz_Delta,F1_Delta,F2_Delta,C1_Delta,C2_Delta,P1_Delta,P2_Delta,AF3_Delta,AF4_Delta,FC3_Delta,FC4_Delta,CP3_Delta,CP4_Delta,PO3_Delta,PO4_Delta,F5_Delta,F6_Delta,C5_Delta,C6_Delta,P5_Delta,P6_Delta,AF7_Delta,AF8_Delta,FT7_Delta,FT8_Delta,TP7_Delta,TP8_Delta,PO7_Delta,PO8_Delta,Fpz_Delta,CPz_Delta,POz_Delta,TP10_Delta,Fp1_Theta,Fp2_Theta,F3_Theta,F4_Theta,C3_Theta,C4_Theta,P3_Theta,P4_Theta,O1_Theta,O2_Theta,F7_Theta,F8_Theta,T7_Theta,T8_Theta,P7_Theta,P8_Theta,Fz_Theta,Cz_Theta,Pz_Theta,Oz_Theta,FC1_Theta,FC2_Theta,CP1_Theta,CP2_Theta,FC5_Theta,FC6_Theta,CP5_Theta,CP6_Theta,FT9_Theta,FT10_Theta,FCz_Theta,AFz_Theta,F1_Theta,F2_Theta,C1_Theta,C2_Theta,P1_Theta,P2_Theta,AF3_Theta,AF4_Theta,FC3_Theta,FC4_Theta,CP3_Theta,CP4_Theta,PO3_Theta,PO4_Theta,F5_Theta,F6_Theta,C5_Theta,C6_Theta,P5_Theta,P6_Theta,AF7_Theta,AF8_Theta,FT7_Theta,FT8_Theta,TP7_Theta,TP8_Theta,PO7_Theta,PO8_Theta,Fpz_Theta,CPz_Theta,POz_Theta,TP10_Theta,Fp1_Alpha,Fp2_Alpha,F3_Alpha,F4_Alpha,C3_Alpha,C4_Alpha,P3_Alpha,P4_Alpha,O1_Alpha,O2_Alpha,F7_Alpha,F8_Alpha,T7_Alpha,T8_Alpha,P7_Alpha,P8_Alpha,Fz_Alpha,Cz_Alpha,Pz_Alpha,Oz_Alpha,FC1_Alpha,FC2_Alpha,CP1_Alpha,CP2_Alpha,FC5_Alpha,FC6_Alpha,CP5_Alpha,CP6_Alpha,FT9_Alpha,FT10_Alpha,FCz_Alpha,AFz_Alpha,F1_Alpha,F2_Alpha,C1_Alpha,C2_Alpha,P1_Alpha,P2_Alpha,AF3_Alpha,AF4_Alpha,FC3_Alpha,FC4_Alpha,CP3_Alpha,CP4_Alpha,PO3_Alpha,PO4_Alpha,F5_Alpha,F6_Alpha,C5_Alpha,C6_Alpha,P5_Alpha,P6_Alpha,AF7_Alpha,AF8_Alpha,FT7_Alpha,FT8_Alpha,TP7_Alpha,TP8_Alpha,PO7_Alpha,PO8_Alpha,Fpz_Alpha,CPz_Alpha,POz_Alpha,TP10_Alpha,Fp1_Beta,Fp2_Beta,F3_Beta,F4_Beta,C3_Beta,C4_Beta,P3_Beta,P4_Beta,O1_Beta,O2_Beta,F7_Beta,F8_Beta,T7_Beta,T8_Beta,P7_Beta,P8_Beta,Fz_Beta,Cz_Beta,Pz_Beta,Oz_Beta,FC1_Beta,FC2_Beta,CP1_Beta,CP2_Beta,FC5_Beta,FC6_Beta,CP5_Beta,CP6_Beta,FT9_Beta,FT10_Beta,FCz_Beta,AFz_Beta,F1_Beta,F2_Beta,C1_Beta,C2_Beta,P1_Beta,P2_Beta,AF3_Beta,AF4_Beta,FC3_Beta,FC4_Beta,CP3_Beta,CP4_Beta,PO3_Beta,PO4_Beta,F5_Beta,F6_Beta,C5_Beta,C6_Beta,P5_Beta,P6_Beta,AF7_Beta,AF8_Beta,FT7_Beta,FT8_Beta,TP7_Beta,TP8_Beta,PO7_Beta,PO8_Beta,Fpz_Beta,CPz_Beta,POz_Beta,TP10_Beta,Fp1_Gamma,Fp2_Gamma,F3_Gamma,F4_Gamma,C3_Gamma,C4_Gamma,P3_Gamma,P4_Gamma,O1_Gamma,O2_Gamma,F7_Gamma,F8_Gamma,T7_Gamma,T8_Gamma,P7_Gamma,P8_Gamma,Fz_Gamma,Cz_Gamma,Pz_Gamma,Oz_Gamma,FC1_Gamma,FC2_Gamma,CP1_Gamma,CP2_Gamma,FC5_Gamma,FC6_Gamma,CP5_Gamma,CP6_Gamma,FT9_Gamma,FT10_Gamma,FCz_Gamma,AFz_Gamma,F1_Gamma,F2_Gamma,C1_Gamma,C2_Gamma,P1_Gamma,P2_Gamma,AF3_Gamma,AF4_Gamma,FC3_Gamma,FC4_Gamma,CP3_Gamma,CP4_Gamma,PO3_Gamma,PO4_Gamma,F5_Gamma,F6_Gamma,C5_Gamma,C6_Gamma,P5_Gamma,P6_Gamma,AF7_Gamma,AF8_Gamma,FT7_Gamma,FT8_Gamma,TP7_Gamma,TP8_Gamma,PO7_Gamma,PO8_Gamma,Fpz_Gamma,CPz_Gamma,POz_Gamma,TP10_Gamma
dateTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [21]:
full_df_2021.to_pickle("full_df_2021_NOX.pkl")